In [2]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

**Dataset**


In [34]:
data=pd.read_csv(r'C:\Users\messa\Desktop\MIV\Fouille_donn\weather.nominal.csv',header=None)

In [35]:
data.columns=['outlook','temperature','humidity','windy','play']

In [36]:
data

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


**Support**

In [27]:
data["outlook"]="outlook"+"="+data["outlook"]

In [28]:
data

,outlook,temperature,humidity,windy,play
0,outlook=sunny,hot,high,False,no
1,outlook=sunny,hot,high,True,no
2,outlook=overcast,hot,high,False,yes
3,outlook=rainy,mild,high,False,yes
4,outlook=rainy,cool,normal,False,yes
5,outlook=rainy,cool,normal,True,no
6,outlook=overcast,cool,normal,True,yes
7,outlook=sunny,mild,high,False,no
8,outlook=sunny,cool,normal,False,yes
9,outlook=rainy,mild,normal,False,yes


In [7]:

def support(transactions , min_support):
  itemset=set([item for sublist in transactions for item in sublist])
  count_items={}
  for item in itemset :
    count=0
    for transaction in transactions :
      set_t=set(transaction)
      if item in transaction :
        count+=1
        
    support = count/len(transactions)
    #filter items that have support > min_support
    if min_support <= support and support!=1:
      count_items[item]=support
  count_items=pd.DataFrame.from_dict(count_items,orient='index')
  return count_items


In [8]:
import itertools

def create_itemsets(items,k):
  # items=support.index
  perm_set = itertools.combinations(items,k)
  return set(perm_set)

In [9]:
def support_itemsets(transactions , itemsets , min_support):
  count_items={}
  for item in itemsets :
    count=0
    i_set=set(item)
    for transaction in transactions :
      t_set=set(transaction)
      if t_set.intersection(i_set)==i_set :
        count+=1
    support = count/len(transactions)
    #filter items that have support > min_support
    if min_support <= support and support!=1:
       count_items[item]=support
  count_items=pd.DataFrame.from_dict(count_items,orient='index')
  return count_items

**Apriori**

In [52]:
def apriori(df,min_support):
  transactions_new = data.copy()
  for c in transactions_new .columns:
      transactions_new[c]=str(c)+"="+transactions_new[c].apply(str)
  transactions=transactions_new.to_numpy()
  item_sep=support(transactions,min_support)
  #min sup >0 a faire
  k=2
  itemset=item_sep.index
  df=pd.DataFrame()
  while len(item_sep)!=0:
    itemsets=create_itemsets(itemset,k)
    item_sep=support_itemsets(transactions,itemsets,min_support)
    print(item_sep,k)

    if not item_sep.empty:
        itemset=list()
        for i in item_sep.index:
         for g in i:
            itemset.append(g)
        itemset=set(itemset)
    k+=1
  
  it=len(itemset)+1
  for i in range(1,it):
    itemsets=create_itemsets(itemset,i)
    item_sep=support_itemsets(transactions,itemsets,min_support)
    df=pd.concat([item_sep,df])
  df=df.reset_index()
  df.columns=["itemsets","support"]
 
  return df,itemset


In [11]:
d=data.to_numpy()

In [53]:
dk,i=apriori(data,0.4)

                                    0
(play=yes, windy=False)      0.428571
(play=yes, humidity=normal)  0.428571 2
Empty DataFrame
Columns: []
Index: [] 3


In [39]:
transactions_new = data.copy()
for c in transactions_new .columns:
    transactions_new[c]=str(c)+"="+transactions_new[c].apply(str)

In [40]:
transactions_new

,outlook,temperature,humidity,windy,play
0,outlook=sunny,temperature=hot,humidity=high,windy=False,play=no
1,outlook=sunny,temperature=hot,humidity=high,windy=True,play=no
2,outlook=overcast,temperature=hot,humidity=high,windy=False,play=yes
3,outlook=rainy,temperature=mild,humidity=high,windy=False,play=yes
4,outlook=rainy,temperature=cool,humidity=normal,windy=False,play=yes
5,outlook=rainy,temperature=cool,humidity=normal,windy=True,play=no
6,outlook=overcast,temperature=cool,humidity=normal,windy=True,play=yes
7,outlook=sunny,temperature=mild,humidity=high,windy=False,play=no
8,outlook=sunny,temperature=cool,humidity=normal,windy=False,play=yes
9,outlook=rainy,temperature=mild,humidity=normal,windy=False,play=yes


In [54]:
l=[]
for k in dk.itemsets.to_list():
   l.append(set(k))
dk.itemsets=l

In [55]:
dk

,itemsets,support
0,"{play=yes, humidity=normal}",0.428571
1,"{windy=False, play=yes}",0.428571
2,{play=yes},0.642857
3,{windy=False},0.571429
4,{humidity=normal},0.500000


In [56]:
i

{'humidity=normal', 'play=yes', 'windy=False'}

Association Rules

In [57]:
def  Association_Rules(dd,apriori_output,threshold=1,metric='confidence'):
    result=pd.DataFrame(columns=['cause','consequence','confidence','lift'])
    perm_set = itertools.permutations(apriori_output,2)
    l=[]
    for j in perm_set:
        l.append(tuple(j))
    np_avant=[]
    np_apres=[]
    conf=[]
    lift=[]
    perm=l
    for p in perm:
        taille=len(p)-1
        while taille>0:
            if (dd['itemsets']==set(p)).any():
                c1=dd[dd['itemsets']==set(p)].support.values/dd[dd['itemsets']==set(p[-taille:])].support.values
                l1=float(c1)/dd[dd['itemsets']==set(p[:-taille])].support.values
                np_avant.append(set(p[-taille:]))
                np_apres.append(set(p[:-taille]))
                conf.append(c1)
                lift.append((l1))
            taille-=1
    result.cause=np_avant
    result.consequence=np_apres
    result.confidence=conf
    result.lift=lift
    if metric=="confidence":
     return result.sort_values(by="confidence")
    else:
       return result.sort_values(by="lift")


In [58]:
Association_Rules(dk,i,threshold=1,metric='confidence')

,cause,consequence,confidence,lift
0,{play=yes},{windy=False},[0.6666666666666666],[1.1666666666666667]
3,{play=yes},{humidity=normal},[0.6666666666666666],[1.3333333333333333]
1,{windy=False},{play=yes},[0.75],[1.1666666666666665]
2,{humidity=normal},{play=yes},[0.8571428571428571],[1.3333333333333333]


In [ ]:
#https://www.softwaretestinghelp.com/apriori-algorithm/